In [33]:
!git clone git@ssh.dev.azure.com:v3/uitopensrc/UIT/MLops-AI

Cloning into 'MLops-AI'...
Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


In [1]:
import numpy as np
import pandas as pd
import glob
import time
import pandas as pd

import os
import matplotlib.pyplot as plt
import keras

import cv2

**Lấy dữ liệu**

In [2]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dectection-XXS-using CNN/XSS_dataset.csv', encoding='utf-8-sig')
df.head(10)

,Unnamed: 0,Sentence,Label
0,0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0
1,1,"<tt onmouseover=""alert(1)"">test</tt>",1
2,2,"\t </span> <span class=""reference-text"">Steeri...",0
3,3,"\t </span> <span class=""reference-text""><cite ...",0
4,4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0
5,5,"<li id=""cite_note-118""><span class=""mw-cite-ba...",0
6,6,"<li><a href=""/wiki/Contextualism"" title=""Conte...",0
7,7,"<li id=""cite_note-Representing_causation-95""><...",0
8,8,"<tr><td class=""plainlist"" style=""padding:0 0....",0
9,9,\t </span>,0


In [3]:
df=df[df.columns[-2:]]
df.head()

,Sentence,Label
0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0
1,"<tt onmouseover=""alert(1)"">test</tt>",1
2,"\t </span> <span class=""reference-text"">Steeri...",0
3,"\t </span> <span class=""reference-text""><cite ...",0
4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0


**Tiền Xử Lý Dữ Liệu**

In [4]:
# Get Sentences data from data frame
sentences=df['Sentence'].values
sentences[1]

'<tt onmouseover="alert(1)">test</tt>'

In [5]:
print(len(sentences))

13686


**Vector Hóa Dữ liệu văn bản**

In [6]:
# Convert to ASCII

def convert_to_ascii(sentence):
    sentence_ascii=[]

    for i in sentence:

        if(ord(i)<8222):      # ” has ASCII of 8221

            if(ord(i)==8217): # ’  :  8217
                sentence_ascii.append(134)


            if(ord(i)==8221): # ”  :  8221
                sentence_ascii.append(129)

            if(ord(i)==8220): # “  :  8220
                sentence_ascii.append(130)


            if(ord(i)==8216): # ‘  :  8216
                sentence_ascii.append(131)

            if(ord(i)==8217): # ’  :  8217
                sentence_ascii.append(132)

            if(ord(i)==8211): # –  :  8211
                sentence_ascii.append(133)


            """
            If values less than 128 store them else discard them
            """
            if (ord(i)<=128):
                    sentence_ascii.append(ord(i))

            else:
                    pass


    zer=np.zeros((10000))

    for i in range(len(sentence_ascii)):
        zer[i]=sentence_ascii[i]

    zer.shape=(100, 100)


#     plt.plot(image)
#     plt.show()
    return zer

In [7]:
# send each sentence to be converted to ASCII


arr=np.zeros((len(sentences),100,100))

for i in range(len(sentences)):

    image=convert_to_ascii(sentences[i])

    x=np.asarray(image,dtype='float')
    image =  cv2.resize(x, dsize=(100,100), interpolation=cv2.INTER_CUBIC)
    image/=128


#     if i==1:
#         plt.plot(image)
#         plt.show()
    arr[i]=image

In [8]:
print("Input data shape : ", arr.shape)

Input data shape :  (13686, 100, 100)


In [9]:
# Reshape data for input to CNN
data = arr.reshape(arr.shape[0], 100, 100, 1)

In [10]:
data.shape

(13686, 100, 100, 1)

In [11]:
y=df['Label'].values

**Xây dựng Model**

In [12]:
# Split into train/test data
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(data,y, test_size=0.2, random_state=42)

In [13]:
# import libraries for Making Model
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D,Flatten,Dropout,MaxPool2D, BatchNormalization

**Tối Ưu Hóa Mô Hình:**

Dropout giúp tránh overfitting bằng cách ngẫu nhiên loại bỏ một số kết nối trong quá trình huấn luyện. Batch Normalization giúp ổn định và tăng tốc độ hội tụ của mô hình.

In [14]:
# A basic CNN Model
# Number of layers = 11
# Number of Convolutional layer: 3

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu, input_shape=(100,100,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [15]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2  (None, 49, 49, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 23, 23, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 256)       295168    
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 10, 10, 256)       0

In [16]:
# Stop when validation accuracy > 97

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.97):
            print(" \n Reached 97% + validation accuracy")
            self.model.stop_training=True
callbacks = myCallback()

In [17]:
batch_size = 64
num_epoch = 8
#model training
model_log = model.fit(trainX, trainY,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1,
          validation_data=( testX,  testY)
#                        callbacks=[callbacks]
                     )

Epoch 1/8
172/172 [==============================] - 695s 4s/step - loss: 0.3178 - accuracy: 0.8601 - val_loss: 0.6341 - val_accuracy: 0.4602
Epoch 2/8
172/172 [==============================] - 646s 4s/step - loss: 0.0799 - accuracy: 0.9744 - val_loss: 0.4552 - val_accuracy: 0.7714
Epoch 3/8
172/172 [==============================] - 684s 4s/step - loss: 0.0701 - accuracy: 0.9773 - val_loss: 0.0914 - val_accuracy: 0.9748
Epoch 4/8
172/172 [==============================] - 681s 4s/step - loss: 0.0478 - accuracy: 0.9854 - val_loss: 0.0437 - val_accuracy: 0.9861
Epoch 5/8
172/172 [==============================] - 636s 4s/step - loss: 0.0397 - accuracy: 0.9865 - val_loss: 0.0434 - val_accuracy: 0.9828
Epoch 6/8
172/172 [==============================] - 677s 4s/step - loss: 0.0324 - accuracy: 0.9890 - val_loss: 0.0768 - val_accuracy: 0.9777
Epoch 7/8
172/172 [==============================] - 675s 4s/step - loss: 0.0347 - accuracy: 0.9891 - val_loss: 0.0273 - val_accuracy: 0.9890
Epoch 

In [18]:
# predict for test set
pred=model.predict(testX)

86/86 [==============================] - 44s 508ms/step


In [19]:
# Threshold values predicted

for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0

In [20]:
# Count True predicted and wrong predicted

true=0
false=0

for i in range(len(pred)):
    if pred[i] == testY[i]:
        true+=1
    else:
        false+=1

print("correct predicted :: ", true)
print("false prediction :: ", false)

correct predicted ::  2713
false prediction ::  25


In [21]:
# Number of attack and benign data in test set

attack=0
benign=0
for i in range(len(testY)):
    if testY[i]==1:
        attack+=1
    else:
        benign+=1

print("Attack data in test set :: ", attack)
print(" Benign data in test set :: ", benign)

Attack data in test set ::  1478
 Benign data in test set ::  1260


In [22]:
# Takes True positive, true negative, false positive and false negative as input
# Returns Accuracy

def accuracy_function(tp,tn,fp,fn):

    accuracy = (tp+tn) / (tp+tn+fp+fn)

    return accuracy

In [23]:
# Takes True Positive and false positive
# Returns Precision

def precision_function(tp,fp):

    precision = tp / (tp+fp)

    return precision

In [24]:
#Takes True Positive and false negative
# Returns Recall

def recall_function(tp,fn):

    recall=tp / (tp+fn)

    return recall

In [25]:
#Takes true values and predicted
# Returns Accuracy, precision and recall


def confusion_matrix(truth,predicted):

    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0

    for true,pred in zip(truth,predicted):
        if true == 1:
            if pred == true:
                true_positive += 1
            elif pred != true:
                false_negative += 1

        elif true == 0:
            if pred == true:
                true_negative += 1
            elif pred != true:
                false_positive += 1

    accuracy=accuracy_function(true_positive, true_negative, false_positive, false_negative)
    precision=precision_function(true_positive, false_positive)
    recall=recall_function(true_positive, false_negative)

    return (accuracy,
            precision,
           recall)

In [26]:
accuracy,precision,recall=confusion_matrix(testY,pred)

In [27]:
print(" Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

 Accuracy : 0.9908692476260044 
 Precision : 0.9932111337406653 
 Recall : 0.989851150202977


In [30]:
import joblib
joblib.dump(model, '/content/drive/MyDrive/Colab Notebooks/Dectection-XXS-using CNN/model_AI.pkl')

['/content/drive/MyDrive/Colab Notebooks/Dectection-XXS-using CNN/model_AI.pkl']

In [31]:
# Lưu trữ mô hình
model.save('/content/drive/MyDrive/Colab Notebooks/Dectection-XXS-using CNN/xss_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
